This code calculates manageable, vulnerable, and irreplaceable carbon across Y2Y

In [1]:
# import packages
import ee
import geemap

import pandas as pd

In [2]:
# authenticate the EE api
# ee.Authenticate()

In [3]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [4]:
# define EE datasets
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")
# countries = ee.FeatureCollection("USDOS/LSIB/2017")
# y2y_ecoregions = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
# y2y_protected_areas = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_protected_areas")
# y2y_protected_areas_canada = ee.FeatureCollection(
#     "projects/ee-bermane/assets/y2y_protected_areas_canada")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
soc_0_100cm_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_0_30cm_olm = ee.Image("projects/ee-bermane/assets/soc_0_30cm_kg_m2_olm")
soc_30_100cm_olm = ee.Image(
    "projects/ee-bermane/assets/soc_30_100cm_kg_m2_olm")
# soc_soilgrids = ee.Image("projects/soilgrids-isric/ocs_mean")
soc_0_100cm_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_0_30cm_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon30cm_250m_kg-m2_version3")
# soc_socs = ee.Image("projects/ee-bermane/assets/SOCS_0_1m_mg_ha_year_2010")
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

## Create landcover/biome masks

Landcover key:
* 1: "Temperate or sub-polar needleleaf forest",
* 2: "Sub-polar taiga needleleaf forest",
* 3: "Tropical or sub-tropical broadleaf evergreen forest",
* 4: "Tropical or sub-tropical broadleaf deciduous forest",
* 5: "Temperate or sub-polar broadleaf deciduous forest",
* 6: "Mixed forest",
* 7: "Tropical or sub-tropical shrubland",
* 8: "Temperate or sub-polar shrubland",
* 9: "Tropical or sub-tropical grassland",
* 10: "Temperate or sub-polar grassland",
* 11: "Sub-polar or polar shrubland-lichen-moss",
* 12: "Sub-polar or polar grassland-lichen-moss",
* 13: "Sub-polar or polar barren-lichen-moss",
* 14: "Wetland",
* 15: "Cropland",
* 16: "Barren land",
* 17: "Urban and built-up",
* 18: "Water",
* 19: "Snow and ice"

Biome key:
* 5: Temperate Conifer Forests
* 6: Boreal Forests/Taiga
* 8: Temperate Grasslands, Savannas, and Shrublands
* 11: Tundra
* 13: Deserts and Xeric Shrublands

Peat key:
* 1: Peat dominated
* 2: Peat in soil mosaic

In [5]:
# Rasterize y2y_biomes to use for ecosystem classification based on BIOME_ID:
biomes_raster = y2y_biomes.reduceToImage(
    properties=['BIOME_ID'], reducer=ee.Reducer.first())

# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (5, 8, 13)
temperate_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_forest_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (8)
temperate_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(8))
)

# MONTANE GRASSLANDS AND SHRUBLANDS --> AGRICULTURE
# create a mask based on grasslands and shrublands LC type (8, 10)
# and not peatlands (1, 2)
# and biomes mask (5)
montane_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5))
)

# NORTHERN GRASSLANDS AND SHRUBLANDS --> MINING/OIL/GAS/DEV
# create a mask based on grasslands, shrublands, lichen-moss LC type (8, 10, 11, 12, 13)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_grass_shrub_mask = (
    (lc.eq(8)
     .Or(lc.eq(10))
     .Or(lc.eq(11))
     .Or(lc.eq(12))
     .Or(lc.eq(13)))
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE WETLANDS --> AGRICULTURE
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN WETLANDS --> MINING/OIL/GAS/DEV
# create a mask based on wetlands LC type (14)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_wetlands_mask = (
    lc.eq(14)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# TEMPERATE PEATLANDS --> AGRICULTURE
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (5, 8)
temperate_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)

# NORTHERN PEATLANDS --> MINING/OIL/GAS/DEV
# create a mask based on peatlands LC type (1, 2)
# and biomes mask (6, 11)
northern_peatlands_mask = (
    peat.mask()
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# NORTHERN BARREN LANDS --> MINING/OIL/GAS/DEV
# create a mask based on barren lands LC type (16)
# and not peatlands (1, 2)
# and biomes mask (6, 11)
northern_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)

# check masks to make sure they encompass full extent
mask_check = (
    temperate_forest_mask
    .Or(northern_forest_mask)
    .Or(temperate_grass_shrub_mask)
    .Or(montane_grass_shrub_mask)
    .Or(northern_grass_shrub_mask)
    .Or(temperate_wetlands_mask)
    .Or(northern_wetlands_mask)
    .Or(temperate_peatlands_mask)
    .Or(northern_peatlands_mask)
    .Or(northern_barren_mask)
)

# Non-manageable systems
# DESERT --> NOT MANAGEABLE
# biomes mask (13)
# and not peatlands (1, 2)
# and not forest (1-6)
desert_mask = (
    biomes_raster.eq(13)
    .And(peat.mask().Not())
    .And(lc.gte(7))
)

# TEMPERATE BARREN LANDS --> NOT MANAGEABLE
# LC type (16)
# and not peatlands (1, 2)
# and biomes mask (5, 8)
temperate_barren_mask = (
    lc.eq(16)
    .And(peat.mask().Not())
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8)))
)


# Vegetation cover for biomass replaceability in peatlands
# TEMPERATE FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (5, 8, 13)
temperate_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(5)
         .Or(biomes_raster.eq(8))
         .Or(biomes_raster.eq(13)))
)

# NORTHERN FORESTS --> FORESTRY
# create a mask based on forest lc types
# and biomes mask (6, 11)
northern_forest_bio_mask = (
    lc.gte(1)
    .And(lc.lte(6))
    .And(biomes_raster.eq(6)
         .Or(biomes_raster.eq(11)))
)


## Calculate manageable biomass stock from ESA CCI data

Includes all biomass except for non-forest pixels within the desert biome and the following LC types: barren ground, snow/ice, water, cropland, urban and built-up.

## NEED TO MASK PROTECTED AREAS!

In [6]:
# calc 2021 ESA CCI biomass
# grab the 2021 AGB images
agb_2021 = agb.filter(ee.Filter.stringContains("system:index", "2021")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# mask litter only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest

litter_mask = lc.gte(1).And(lc.lte(6))

litter = agb_2021.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb_2021.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio_2021 = agb_2021.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask barren ground, snow/ice, water, cropland, urban and built-up
bio_2021 = bio_2021.multiply(0.47).updateMask(
    lc.neq(18).And(lc.neq(19)).And(lc.neq(16)).And(lc.neq(15)).And(lc.neq(17)))

# mask desert pixels from bio_2021
bio_2021 = bio_2021.updateMask(desert_mask.Not())

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio_2021.select(['agb_t_ha']).mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# # calculate total biomass c per pixel and rename bands
# bio_stock_2021 = bio_2021.multiply(
#     pixel_area_ha
# ).rename(
#     ['agb_t', 'bgb_t']
# ).addBands(
#     pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
# ).addBands(
#     pixel_area_ha.rename(['pixel_area_ha'])
# )

# calculate biomass stock
bio_2021_stock = (
    bio_2021
    .multiply(pixel_area_ha)
    .rename(['agb_t', 'bgb_t'])
)

# combine bands for manageable biomass density and stock
manageable_bio_density = (
    bio_2021
    .select(['agb_t_ha', 'bgb_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_biomass_t_ha')
)

manageable_bio_stock = (
    bio_2021_stock
    .select(['agb_t', 'bgb_t'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_biomass_t')
)

## Calculate manageable soil carbon stock from Sothe Canada data and Open Land Map US data

Includes all SOC 0-3cm and 30-100cm except for non-forest pixels within the desert biome, barren lands within temperate zones, and the following LC types: snow/ice, water, cropland, urban and built-up.

## NEED TO MASK PROTECTED AREAS!!!

In [7]:
# calc open land map soc for 0-30cm and 30-100cm
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_0_30cm_olm_t_ha = soc_0_30cm_olm.multiply(10).rename(
    'soc_0_30cm_t_ha')

soc_30_100cm_olm_t_ha = soc_30_100cm_olm.multiply(10).rename(
    'soc_30_100cm_t_ha')

# add both to same image
soc_olm_t_ha = soc_0_30cm_olm_t_ha.addBands(soc_30_100cm_olm_t_ha)

# resample to match sothe to blend images
soc_olm_t_ha_reproj_sothe = soc_olm_t_ha.resample('bilinear').toFloat()

In [8]:
# calc sothe soc for 0-30cm and 30-100cm
# multiply by 10 to get t/ha
# mask water/snow/ice/cropland, urban and built-up
soc_0_30cm_sothe_t_ha = soc_0_30cm_sothe.multiply(10).rename(
    'soc_0_30cm_t_ha')

soc_0_100cm_sothe_t_ha = soc_0_100cm_sothe.multiply(10).rename(
    'soc_0_100cm_t_ha')

soc_30_100cm_sothe_t_ha = soc_0_100cm_sothe_t_ha.subtract(soc_0_30cm_sothe_t_ha).rename(
    'soc_30_100cm_t_ha')

# add both to same image
soc_sothe_t_ha = soc_0_30cm_sothe_t_ha.addBands(soc_30_100cm_sothe_t_ha)

# blend sothe and olm carbon across y2y
# band names must match to mosaic!
soc_blend = ee.ImageCollection([soc_olm_t_ha_reproj_sothe, soc_sothe_t_ha]).mosaic().reproject(
    crs=soc_sothe_t_ha.projection(),
    crsTransform=soc_sothe_t_ha.projection().getInfo().get('transform')
    # mask snow/ice, water, cropland, urban and built-up
).updateMask(lc.neq(18).And(lc.neq(19)).And(lc.neq(15)).And(lc.neq(17)))

# mask desert and temperate barren pixels from soc_blend
soc_blend = (
    soc_blend
    .updateMask(desert_mask.Not())
    .updateMask(temperate_barren_mask.Not())
)

# apply masks to soc_blend 30-100cm band ONLY
updated_30_100cm_soc = (
    soc_blend
    .select('soc_30_100cm_t_ha')
    .updateMask(temperate_forest_mask.Not())
    .updateMask(northern_forest_mask.Not())
    .updateMask(temperate_grass_shrub_mask.Not())
    .updateMask(northern_grass_shrub_mask.Not())
    .updateMask(northern_wetlands_mask.Not())
    .updateMask(northern_peatlands_mask.Not())
    .updateMask(northern_barren_mask.Not())
)

# add masked 30-100cm soc back to soc_blend
soc_blend = soc_blend.addBands(updated_30_100cm_soc, overwrite=True)

# Create soc layer mask to filter pixel area raster
# use 0-30cm soc band since masking more pixels out of 30-100cm band
soc_mask = soc_blend.select('soc_0_30cm_t_ha').mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# # multiply by pixel area to get total carbon per pixel
# soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
#     pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
# ).addBands(
#     pixel_area_ha.rename(['pixel_area_ha'])
# )

# calculate soc stock
soc_blend_stock = (
    soc_blend
    .multiply(pixel_area_ha)
    .rename('soc_0_30cm_t', 'soc_30_100cm_t')
)

# combine bands for manageable soc density and stock
manageable_soc_density = (
    soc_blend
    .select(['soc_0_30cm_t_ha', 'soc_30_100cm_t_ha'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_soc_t_ha')
)

manageable_soc_stock = (
    soc_blend_stock
    .select(['soc_0_30cm_t', 'soc_30_100cm_t'])
    .reduce(ee.Reducer.sum())
    .rename('manageable_soc_t')
)

## Calculate vulnerable biomass carbon stock
All manageable biomass carbon is vulnerable, so no calculations needed.

In [9]:
# rename total vulnerable biomass carbon density and stock
vulnerable_bio_density = manageable_bio_density.rename('vulnerable_biomass_t_ha')

vulnerable_bio_stock = manageable_bio_stock.rename('vulnerable_biomass_t')

## Calculate vulnerable SOC stock
Using the seismic line values for vulnerable carbon in northern systems

In [10]:
# set vulnerable SOC based on different masks
# temperate_forest_mask --> 0%
# northern_forest_mask --> 0%
# temperate_grass_shrub_mask --> 36.1%
# montane_grass_shrub_mask --> 36.1%
# northern_grass_shrub_mask --> 32.2%
# temperate_wetlands_mask --> 42%
# northern_wetlands_mask --> 32.2%
# temperate_peatlands_mask --> 100% or 135 t/ha
# northern_peatlands_mask --> 32.2%
# northern_barren_mask --> 32.2%
vulnerable_soc_density = (
    manageable_soc_density
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, manageable_soc_density.multiply(.361))
    .where(montane_grass_shrub_mask, manageable_soc_density.multiply(.361))
    .where(northern_grass_shrub_mask, manageable_soc_density.multiply(.322))
    .where(temperate_wetlands_mask, manageable_soc_density.multiply(.42))
    .where(northern_wetlands_mask, manageable_soc_density.multiply(.322))
    .where(temperate_peatlands_mask.And(manageable_soc_density.gte(135)), 135)
    .where(northern_peatlands_mask, manageable_soc_density.multiply(.322))
    .where(northern_barren_mask, manageable_soc_density.multiply(.322))
    .rename('vulnerable_soc_t_ha')
)

# calculate vulnerable soc stock
vulnerable_soc_stock = (
    vulnerable_soc_density
    .multiply(pixel_area_ha)
    .rename('vulnerable_soc_t')
)

## Calculate recoverable biomass carbon stock
Based on 30 year recovery

In [11]:
# redo masks to not include peat since biomass isn't affected by peat
# peat areas take on values of LC types

# set recoverable biomass C based on different masks
# temperate_forest_bio_mask --> 1.658*20 + 2.127*10 t/ha
# northern_forest_bio_mask --> 1.268*20 + 0.655*10 t/ha
# everywhere else is just the same amount as vulnerable
recoverable_bio_density = (
    vulnerable_bio_density
    .where(temperate_forest_bio_mask, 1.658*20 + 2.127*10)
    .where(northern_forest_bio_mask, 1.268*20 + 0.655*10)
    .rename('recoverable_biomass_t_ha')
)

# set recoverable value ceiling at vulnerable values
recoverable_bio_density = recoverable_bio_density.where(
    recoverable_bio_density.gt(vulnerable_bio_density), vulnerable_bio_density)

# calculate recoverable biomass stock
recoverable_bio_stock = (
    recoverable_bio_density
    .multiply(pixel_area_ha)
    .rename('recoverable_biomass_t')
)

## Calculate recoverable SOC stock

In [12]:
# calculate remaining soc density after land conversion
remaining_soc_density = (
    manageable_soc_density
    .subtract(vulnerable_soc_density)
)

# set recoverable SOC based on different masks
# temperate_forest_mask --> none vulnerable
# northern_forest_mask --> none vulnerable
# temperate_grass_shrub_mask --> 57.3% of remaining added back
# montane_grass_shrub_mask --> 25% of remaining added back
# northern_grass_shrub_mask --> 1.3*30 t/ha
# temperate_wetlands_mask --> 1.226*30 t/ha
# northern_wetlands_mask --> 1.226*30 t/ha
# temperate_peatlands_mask --> 0.28*30 t/ha
# northern_peatlands_mask --> 0.21*30 t/ha
# northern_barren_mask --> 1.3*30 t/ha
recoverable_soc_density = (
    remaining_soc_density
    .where(temperate_forest_mask, 0)
    .where(northern_forest_mask, 0)
    .where(temperate_grass_shrub_mask, remaining_soc_density.multiply(.573))
    .where(montane_grass_shrub_mask, remaining_soc_density.multiply(.25))
    .where(northern_grass_shrub_mask, 1.3*30)
    .where(temperate_wetlands_mask, 1.226*30)
    .where(northern_wetlands_mask, 1.226*30)
    .where(temperate_peatlands_mask, 0.28*30)
    .where(northern_peatlands_mask, 0.21*30)
    .where(northern_barren_mask, 1.3*30)
    .rename('recoverable_soc_t_ha')
)

# set recoverable value ceiling at vulnerable values
recoverable_soc_density = recoverable_soc_density.where(
    recoverable_soc_density.gt(vulnerable_soc_density), vulnerable_soc_density)

# calculate recoverable soc stock
recoverable_soc_stock = (
    recoverable_soc_density
    .multiply(pixel_area_ha)
    .rename('recoverable_soc_t')
)

## Calculate Irrecoverable biomass C and SOC
Vulnerable minus recoverable

In [13]:
# Irrecoverable biomass C
irrecoverable_bio_density = (
    vulnerable_bio_density
    .subtract(recoverable_bio_density)
    .rename('irrecoverable_biomass_t_ha')
)

# calculate irrecoverable biomass stock
irrecoverable_bio_stock = (
    irrecoverable_bio_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_biomass_t')
)

# irrecoverable SOC
irrecoverable_soc_density = (
    vulnerable_soc_density
    .subtract(recoverable_soc_density)
    .rename('irrecoverable_soc_t_ha')
)

# calculate irrecoverable soc stock
irrecoverable_soc_stock = (
    irrecoverable_soc_density
    .multiply(pixel_area_ha)
    .rename('irrecoverable_soc_t')
)

In [14]:
# check min and max values of irrecoverable carbon
min_irrecoverable_bio = irrecoverable_bio_density.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=y2y.geometry(),
    scale=irrecoverable_bio_density.projection().nominalScale(),
    maxPixels=1e9)

max_irrecoverable_bio = irrecoverable_bio_density.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=y2y.geometry(),
    scale=irrecoverable_bio_density.projection().nominalScale(),
    maxPixels=1e9)

min_irrecoverable_soc = irrecoverable_soc_density.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=y2y.geometry(),
    scale=irrecoverable_soc_density.projection().nominalScale(),
    maxPixels=1e9)

max_irrecoverable_soc = irrecoverable_soc_density.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=y2y.geometry(),
    scale=irrecoverable_soc_density.projection().nominalScale(),
    maxPixels=1e9)

print('min_irrecoverable_bio:', min_irrecoverable_bio.getInfo())
print('max_irrecoverable_bio:', max_irrecoverable_bio.getInfo())
print('min_irrecoverable_soc:', min_irrecoverable_soc.getInfo())
print('max_irrecoverable_soc:', max_irrecoverable_soc.getInfo())


min_irrecoverable_bio: {'irrecoverable_biomass_t_ha': None}
max_irrecoverable_bio: {'irrecoverable_biomass_t_ha': None}
min_irrecoverable_soc: {'irrecoverable_soc_t_ha': 0}
max_irrecoverable_soc: {'irrecoverable_soc_t_ha': 738.6416778564453}


## Add layers to map

In [21]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [23]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(desert_mask, {}, "Desert Mask")
m.addLayer(y2y_biomes, {}, "Biomes")
m.addLayer(lc.clip(y2y), {"min": 1, "max": 19,
                          "palette": lc_palette}, 'Landcover')
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(manageable_bio_stock.select(['agb_t']).clip(y2y), {
#     "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB')
# m.addLayer(soc_blend.select(['soc_0_30cm_t_ha']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "SOC Density Blend 0-30cm")
# m.addLayer(soc_blend.select(['soc_30_100cm_t_ha']).clip(y2y), {"min": 1, "max": 450,
#            "palette": biomass_palette}, "SOC Density Blend 30-100cm")
# m.addLayer(pixel_area_soc_extent.clip(y2y), {}, 'Carbon Mask')
# m.addLayer(peat.clip(y2y), {}, "Peat Distribution")
# m.addLayer(rra, {}, "RRA")
# m.addLayer(vulnerable_soc_dens.select('soc_0_30cm_t_ha').clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Density 0-30cm")
# m.addLayer(vulnerable_soc_dens.select('soc_30_100cm_t_ha').clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Density 30-100cm")

# final stock layers
# m.addLayer(manageable_bio_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Manageable Bio Density")
# m.addLayer(manageable_soc_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Manageable SOC Density")
# m.addLayer(vulnerable_bio_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable Bio Density")
# m.addLayer(vulnerable_soc_density.clip(y2y), {
#            "min": 0, "max": 450, "palette": biomass_palette}, "Vulnerable SOC Density")
m.addLayer(irrecoverable_bio_density.clip(y2y), {
           "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable Bio Density")
m.addLayer(irrecoverable_soc_density.clip(y2y), {
           "min": 0, "max": 400, "palette": biomass_palette}, "Irrecoverable SOC Density")


# check masks
# m.addLayer(mask_check.clip(y2y), {}, "Mask Check")
# m.addLayer(temperate_forest_mask.clip(y2y), {}, "Temperate Forest Mask")
# m.addLayer(temperate_grass_shrub_mask.clip(y2y), {}, "Temperate Grasslands and Shrublands")
# m.addLayer(montane_grass_shrub_mask.clip(y2y), {}, "Montane Grasslands and Shrublands")
# m.addLayer(northern_grass_shrub_mask.clip(y2y), {}, "Northern Grasslands and Shrublands")
# m.addLayer(temperate_wetlands_mask.clip(y2y), {}, "Temperate Wetlands")
# m.addLayer(northern_wetlands_mask.clip(y2y), {}, "Northern Wetlands")
# m.addLayer(temperate_peatlands_mask.clip(y2y), {}, "Temperate Peatlands")
# m.addLayer(northern_peatlands_mask.clip(y2y), {}, "Northern Peatlands")
# m.addLayer(northern_barren_mask.clip(y2y), {}, "Northern Barren Lands")
# m.addLayer(desert_mask.clip(y2y), {}, "Desert Mask")

# display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Initial amount
initial_amount = 100

# Loss rate per year (5%)
loss_rate = 0.05

# Number of years
years = 30

# Formula to calculate remaining amount after 30 years with a 5% annual loss
remaining_amount = initial_amount * (1 - loss_rate) ** years
print(remaining_amount)

In [4]:
2146/10000